## **Sentimen Analisis Menggunakan SVM**

Nama : Gevira Zahra Shofa

NIM :  1227050050

Kelas : Prak. Pembelajaran Mesin C

Dataset : https://www.kaggle.com/competitions/si650winter11/data?select=training.txt

Import Library

In [14]:
%matplotlib inline
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import _pickle as cPickle
from scipy.io import loadmat
from sklearn.svm import SVC
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer,TfidfTransformer
# Import train_test_split, StratifiedKFold, and cross_val_score from sklearn.model_selection
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, f1_score, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
# Import GridSearchCV from sklearn.model_selection
from sklearn.model_selection import GridSearchCV
# Import learning_curve from sklearn.model_selection (or use learning_curve from sklearn.model_selection)
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
df=pd.read_csv("training.txt",sep="\t", names=['liked','text'],encoding="utf-8");
df.head(3)

,liked,text
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.


In [16]:
print(len(df))

6918


In [17]:
df.groupby('liked').describe()

text                                  
      count unique                   top freq
liked                                        
0      2975    559  I hate Harry Potter.   85
1      3943    732  I love Harry Potter.  167

Data Preprocessing

In [18]:
def tokens(review):
    return TextBlob(review).words

In [23]:
df.head().text.apply(tokens)

,text
0,"[The, Da, Vinci, Code, book, is, just, awesome]"
1,"[this, was, the, first, clive, cussler, i, 've..."
2,"[i, liked, the, Da, Vinci, Code, a, lot]"
3,"[i, liked, the, Da, Vinci, Code, a, lot]"
4,"[I, liked, the, Da, Vinci, Code, but, it, ulti..."


In [26]:
TextBlob("ready was not a good movie").tags
#nltk.help.upenn_tagset('JJ')

[('ready', 'NN'),
 ('was', 'VBD'),
 ('not', 'RB'),
 ('a', 'DT'),
 ('good', 'JJ'),
 ('movie', 'NN')]

In [29]:
def to_lemmas(review):
    wordss = TextBlob(review.lower()).words
    # for each word, take its "base form" = lemma
    return [word.lemma for word in wordss]

df.text.head().apply(to_lemmas)

,text
0,"[the, da, vinci, code, book, is, just, awesome]"
1,"[this, wa, the, first, clive, cussler, i, 've,..."
2,"[i, liked, the, da, vinci, code, a, lot]"
3,"[i, liked, the, da, vinci, code, a, lot]"
4,"[i, liked, the, da, vinci, code, but, it, ulti..."


In [30]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lmtzr.lemmatize('octopi')
#nltk

'octopus'

Converting text data into vectors

In [31]:
bow_transformer = CountVectorizer(analyzer=to_lemmas).fit(df['text'])
print(len(bow_transformer.vocabulary_))

2113


In [32]:
review1=df['text'][3]
print(review1)
#to check 3rd document/review in collection/database

i liked the Da Vinci Code a lot.


In [33]:
bow=bow_transformer.transform([review1])
print(bow)
bow.shape

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 8 stored elements and shape (1, 2113)>
  Coords	Values
  (0, 43)	1
  (0, 370)	1
  (0, 458)	1
  (0, 949)	1
  (0, 1121)	1
  (0, 1150)	1
  (0, 1837)	1
  (0, 1976)	1


(1, 2113)

In [35]:
# print(bow_transformer.get_feature_names()[372])
# to check 372nd word in collection

# Use get_feature_names_out() instead of the deprecated get_feature_names()
print(bow_transformer.get_feature_names_out()[372])

code-illustrated


In [36]:
review_bow = bow_transformer.transform(df['text'])
print( 'sparse matrix shape:', review_bow.shape)
print('number of non-zeros:', review_bow.nnz) #learn this
print( 'sparsity: %.2f%%' % (100.0 * review_bow.nnz))

sparse matrix shape: (6918, 2113)
number of non-zeros: 71215
sparsity: 7121500.00%


Tf-idf Vectorizer

In [37]:
tfidf_transformer =TfidfTransformer().fit(review_bow)
review_tfidf = tfidf_transformer.transform(review_bow)
review_tfidf.shape

(6918, 2113)

In [38]:
text_train, text_test, liked_train, liked_test = train_test_split(df['text'], df['liked'], test_size=0.2)
print(len(text_train), len(text_test), len(text_train) , len(text_test))

5534 1384 5534 1384


In [39]:
pipeline_svm = Pipeline([
    ('bow', CountVectorizer(analyzer=to_lemmas)),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC()),
])

In [40]:
# pipeline parameters to automatically explore and tune
param_svm = [
  {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
  {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf']},
]

In [43]:
# pipeline parameters to automatically explore and tune
param_svm = [
  {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
  {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf']},
]

grid_svm = GridSearchCV(
    pipeline_svm, #object used to fit the data
    param_grid=param_svm,
    refit=True,  # fit using all data, on the best detected classifier
    n_jobs=-1,  # number of cores to use for parallelization; -1 for "all cores" i.e. to run on all CPUs
    scoring='accuracy',#optimizing parameter
    cv=StratifiedKFold(n_splits=5), # Changed n_folds to n_splits
)

In [45]:
%time classifier = grid_svm.fit(text_train, liked_train) # find the best combination from param_svm
# print(classifier.grid_scores_) # Deprecated and removed
print(classifier.cv_results_) # Use cv_results_ instead

CPU times: user 2.94 s, sys: 270 ms, total: 3.21 s
Wall time: 2min 20s
{'mean_fit_time': array([3.47131343, 2.44757524, 2.44659438, 2.21394   , 6.12832675,
       6.34487853, 4.4318399 , 5.77234859, 2.797752  , 4.36692185,
       2.21682758, 2.78843131]), 'std_fit_time': array([0.61564034, 0.60011463, 0.30654218, 0.19974586, 0.740831  ,
       0.85389202, 0.59054709, 0.5671194 , 0.39718617, 0.41737702,
       0.02889   , 0.64592037]), 'mean_score_time': array([0.69744344, 0.52840061, 0.69903908, 0.52115889, 1.06981726,
       1.07356329, 1.05262346, 1.20882587, 0.75210199, 1.14307027,
       0.55447941, 0.58335814]), 'std_score_time': array([0.17291838, 0.01413354, 0.20983649, 0.00886755, 0.03268877,
       0.04237087, 0.17571264, 0.31495869, 0.22790381, 0.34534346,
       0.02570316, 0.15247289]), 'param_classifier__C': masked_array(data=[1, 10, 100, 1000, 1, 1, 10, 10, 100, 100, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False,
                  

In [46]:
print(classification_report(liked_test, classifier.predict(text_test)))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       583
           1       0.99      0.99      0.99       801

    accuracy                           0.99      1384
   macro avg       0.99      0.99      0.99      1384
weighted avg       0.99      0.99      0.99      1384



In [47]:
print(classifier.predict(["the vinci code is awesome"])[0])

1


In [48]:
print(classifier.predict(["the vinci code is bad"])[0])

0


In [49]:
def gaussKernel(x1, x2, sigma):
    ss=np.power(sigma,2)
    norm= (x1-x2).T.dot(x1-x2)
    return np.exp(-norm/(2*ss))
x1 = np.array([1, 2, 1])
x2 = np.array([0, 4, -1])
sigma = 2
gaussKernel(x1,x2,sigma)

np.float64(0.32465246735834974)